# ProfessorGPT - Proof Of Concept

<p align="center">
  <img src="./prof.png" alt="prof" width="800px" height="800px">
</p>

AI-driven academic assistant leveraging OpenAI's APIs for interactive tutoring and comprehension assessment

### Workflow

1. *Document Intake & Processing*

2. *Goal Formation*

3. *Interactive Tutoring Session*

4. *Understanding Assessment*

## Load the document and initalize openAI client

In [81]:
from openai import OpenAI

topic = "tardigrades"

with open(f"./{topic}.txt", "r") as file:
    file_content = file.read()

client = OpenAI()

## Text Generation API Wrapper

In [118]:
from openai import OpenAI
class TextGeneration:
    def __init__(self, topic :str ):
        self.client = OpenAI()
        self.topic = topic
    def generate_json(self, user_prompt, system_prompt= f"You are a helpful assistant that knows a lot about {self.topic} and only responds with JSON" ):
        return self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content":system_prompt },
                {"role": "user", "content": f"{user_prompt}" },
            ]
        )
    def generate_text(self, user_prompt, system_prompt= f"You are a helpful assistant that knows a lot about {self.topic}" ):
        return self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content":system_prompt },
                {"role": "user", "content": f"{user_prompt}" },
            ]
        ).choices[0].message.content
tg = TextGeneration()

In [216]:
response = tg.generate_json(user_prompt=file_content + "Create a CSV table outlining key points on the topic, with one column for topic-related questions and another for their corresponding answers.")

### Convert to dataframe 

In [217]:
from pprint import pp
import json
import pandas as pd
document = (json.loads(response.choices[0].json()))
quiz_df = pd.DataFrame(json.loads(document['message']['content'])["table"])
quiz_df["user_answer"] = None
quiz_df["user_is_correct"] = None
quiz_df["feedback"] = None

In [218]:
quiz_df

,question,answer,user_answer,user_is_correct,feedback
0,What are tardigrades?,"Tardigrades are microscopic animals belonging to the phylum Tardigrada, also known as 'water bears' or 'moss piglets.'",None,None,None
1,When were tardigrades discovered?,Tardigrades were discovered in 1773 by Johann August Ephraim Goeze.,None,None,None
2,What are the biological features of tardigrades?,"Tardigrades have a segmented body with four pairs of legs, and they typically range from 0.05 to 1.2 millimeters in length. They have a barrel-shaped body and a bear-like appearance.",None,None,None
3,What is cryptobiosis?,Cryptobiosis is a state in which tardigrades lose almost all their body water content and become metabolically inactive until conditions improve.,None,None,None
4,Where can tardigrades be found?,"Tardigrades can be found in diverse habitats worldwide, ranging from the deepest oceans to the highest mountains, from tropical rainforests to polar regions.",None,None,None
5,How many tardigrade species are known?,"Over 1,300 tardigrade species have been described to date.",None,None,None
6,What is the significance of tardigrades in astrobiology?,Tardigrades serve as models for potential survival mechanisms in harsh extraterrestrial environments.,None,None,None
7,What scientific applications do tardigrades hold potential for?,"Tardigrade proteins may have applications in preserving biological materials, such as organ transplants and vaccine development.",None,None,None


## Summarize the Input Document

In [214]:
summary= tg.generate_text(user_prompt=file_content + f"summarize this in 200 words. include all relevant information and background required to answer the following questions : {str(list(quiz_df['question']))}")                      
pp(summary)

('Tardigrades, also known as "water bears" or "moss piglets," are microscopic '
 'animals belonging to the phylum Tardigrada. They were discovered by Johann '
 'August Ephraim Goeze in 1773 and have since fascinated the scientific '
 'community due to their exceptional ability to survive extreme conditions '
 'that would be fatal to most other organisms.\n'
 '\n'
 'Tardigrades are typically 0.05 to 1.2 millimeters in length and have a '
 'barrel-shaped body with four pairs of legs. They exhibit remarkable survival '
 'abilities, such as withstanding temperatures from near absolute zero '
 '(-458°F) to above boiling point (300°F), extreme pressures, and even the '
 'vacuum of space. They can enter a state called cryptobiosis, in which they '
 'lose almost all their body water content and become metabolically inactive '
 'until conditions improve. Studies have shown that tardigrades can survive '
 'without water for over a decade in this state.\n'
 '\n'
 'Tardigrades are incredibly adapt

## Text To Speech API Wrapper

In [215]:
from pathlib import Path
import os
import playsound

class TextToSpeech:
    def __init__(self):
        self.client = OpenAI()
        self.output_file_path = ""

    def generate_speech(self, input_text, output_file_path):
        self.output_file_path = Path(os.getcwd()) / output_file_path
        response = self.client.audio.speech.create(
            model="tts-1",
            voice="alloy",
            input=input_text
        )
        resp = response.stream_to_file(self.output_file_path)
        playsound.playsound(str(self.output_file_path))
        return resp

    def get_output_file_path(self):
        return self.output_file_path

# summary = "test"
tts= TextToSpeech()
tts.generate_speech(input_text=summary, output_file_path="summary.mp3")


## Speech To Text API Wrapper

In [122]:
import sounddevice as sd
import soundfile as sf
from openai import OpenAI


class SpeechToText:
    def __init__(self):
        self.client = OpenAI()

    def record_audio(self, duration=5):
        # [TODO] make this stop recording after x seconds of silence
        sample_rate = 44100
        channels = 1
        audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=channels)
        sd.wait() # Wait until the recording is finished
        output_file_path = "./answer.wav"
        sf.write(output_file_path, audio, sample_rate)
        return output_file_path

    def transcribe_audio(self, audio_path):
        audio_file = open(audio_path, "rb")
        transcript = self.client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
        return transcript

    def record_and_transcribe_audio(self):
        return self.transcribe_audio(self.record_audio())



stt = SpeechToText()


## Quiz Logic

In [123]:
import time
def quiz_user():
  for i, question in enumerate(quiz_df.question[0:]):
    #First read the question to the user
    generate_speech(input_text=question, output_file_path=f"questions/{i}.mp3")

    #Capture user's answer to the question to our quiz_df
    quiz_df["user_answer"][i] = stt.record_and_transcribe_audio().text

    #Grade the user's response and provide feedback
    grading_prompt = f"is {quiz_df.user_answer[i]} a reasonably acceptable answer to {quiz_df.answer}?"
    quiz_df["user_is_correct"][i] = tg.generate_text(user_prompt= grading_prompt + "answer me with only true if acceptable and false if not")
    time.sleep(1)
    quiz_df["feedback"][i] = tg.generate_text(user_prompt= grading_prompt + "provide 15 words of feedback")


pd.set_option("max_colwidth",None)

## Learning Assessment - Failing Example

In [124]:
quiz_user()
display(quiz_df)

,question,answer,user_answer,user_is_correct,feedback
0,What are tardigrades?,Tardigrades are microscopic animals belonging to the phylum Tardigrada.,Tardigrades are tiny creatures that live in the ocean.,False,"The answer is informative and provides interesting facts about tardigrades' characteristics, habitats, and their potential scientific applications."
1,When were tardigrades discovered?,Tardigrades were discovered in 1773 by Johann August Ephraim Goeze.,1730,False,The information provided is accurate and covers a range of interesting facts about tardigrades. Well done!
2,What is the size of tardigrades?,Tardigrades typically range from 0.05 to 1.2 millimeters in length.,I'm not sure.,False,"The answer is informative, accurate, and provides a good overview of tardigrades and their unique characteristics."
3,What are some survival abilities of tardigrades?,"Tardigrades can survive extreme temperatures, pressures, and even the vacuum of space. They can also enter a state called cryptobiosis, where they lose almost all their body water content and become metabolically inactive.",.,False,Informative and concise. Provides key facts about tardigrades and their significance in various fields of research.
4,Where can tardigrades be found?,"Tardigrades can be found in diverse habitats worldwide, including the deepest oceans, highest mountains, tropical rainforests, and polar regions. They occupy nearly every environment on Earth, with high densities reported in mosses and lichens.",in space.,False,Informative and concise answer providing interesting facts about tardigrades and their significance in astrobiology and biotechnology research.
5,What is the scientific significance of tardigrades?,Tardigrades have implications in astrobiology and biotechnology research. Their ability to withstand extreme conditions can provide insights into survival mechanisms in extraterrestrial environments. Tardigrade proteins might also have applications in preserving biological materials and vaccine development.,I'm not sure.,"False. ""I'm not sure."" is not an acceptable answer to the statement ""Tardigrades are microscopic animals belonging to the phylum Tardigrada.""",The answer is informative and provides interesting facts about tardigrades and their significance in various fields.


## Learning Assessment - Passing Example

In [152]:
quiz_user()
display(quiz_df)

,question,answer,user_answer,user_is_correct,feedback
0,What are tardigrades?,Tardigrades are microscopic animals belonging to the phylum Tardigrada.,Tardigrades are microscopic animals belonging to the,True,Your answer is accurate and provides interesting information about tardigrades and their unique abilities and habitats. Well done!
1,When were tardigrades discovered?,Tardigrades were discovered in 1773 by Johann August Ephraim Goeze.,1773 by Johann August Ephraim Geyser,False,The answer 1773 by Johann August Ephraim Geyser is incorrect. The correct answer is Johann August Ephraim Goeze.
2,What is the size of tardigrades?,Tardigrades typically range from 0.05 to 1.2 millimeters in length.,0.05 to 1.2 millimeters,"False. The statement ""0.05 to 1.2 millimeters"" is not a reasonably acceptable answer to the provided 0.","Yes, 0.05 to 1.2 millimeters is a reasonably acceptable answer to question 2 about tardigrade size."
3,What are some survival abilities of tardigrades?,"Tardigrades can survive extreme temperatures, pressures, and even the vacuum of space. They can also enter a state called cryptobiosis, where they lose almost all their body water content and become metabolically inactive.","Extreme temperatures, pressures, and space.",True,"Yes, ""Extreme temperatures, pressures, and space"" is a reasonable and accurate summary of the information provided. Good job!"
4,Where can tardigrades be found?,"Tardigrades can be found in diverse habitats worldwide, including the deepest oceans, highest mountains, tropical rainforests, and polar regions. They occupy nearly every environment on Earth, with high densities reported in mosses and lichens.",Diverse habitats.,True,"Yes, ""Diverse habitats"" is a reasonably acceptable answer to the statement about tardigrades."
5,What is the scientific significance of tardigrades?,Tardigrades have implications in astrobiology and biotechnology research. Their ability to withstand extreme conditions can provide insights into survival mechanisms in extraterrestrial environments. Tardigrade proteins might also have applications in preserving biological materials and vaccine development.,"vaccine development, biological materials, and extraterrestrial environments.",True,The answer is comprehensive and provides relevant information about tardigrades and their significance in astrobiology and biotechnology.


## Quiz Results

In [158]:
import pandas as pd
quiz2_df = quiz_df.copy()
# Clean and convert the column to booleans
quiz2_df['user_is_correct'] = quiz2_df['user_is_correct'].astype(str).str.strip()
quiz2_df['user_is_correct'] = quiz2_df['user_is_correct'].map({'True': True, 'False': False})

# Drop rows with NaN values (if applicable)
quiz2_df = quiz2_df.dropna(subset=['user_is_correct'])

# Determine if the user passed based on a minimum passing score of 74%
user_score = quiz2_df['user_is_correct'].mean()*100
pass_status = "PASSED" if user_score >= 74 else "FAILED"
min_score_to_pass = 75
display(f"User Score = {user_score}%, minimum score to pass: {min_score_to_pass}%, User has {pass_status} the quiz")


'User Score = 80.0%, minimum score to pass: 75%, User has PASSED the quiz'

In [212]:
quiz2_df

,question,answer,user_answer,user_is_correct,feedback
0,What are tardigrades?,Tardigrades are microscopic animals belonging to the phylum Tardigrada.,Tardigrades are microscopic animals belonging to the,True,Your answer is accurate and provides interesting information about tardigrades and their unique abilities and habitats. Well done!
1,When were tardigrades discovered?,Tardigrades were discovered in 1773 by Johann August Ephraim Goeze.,1773 by Johann August Ephraim Geyser,False,The answer 1773 by Johann August Ephraim Geyser is incorrect. The correct answer is Johann August Ephraim Goeze.
3,What are some survival abilities of tardigrades?,"Tardigrades can survive extreme temperatures, pressures, and even the vacuum of space. They can also enter a state called cryptobiosis, where they lose almost all their body water content and become metabolically inactive.","Extreme temperatures, pressures, and space.",True,"Yes, ""Extreme temperatures, pressures, and space"" is a reasonable and accurate summary of the information provided. Good job!"
4,Where can tardigrades be found?,"Tardigrades can be found in diverse habitats worldwide, including the deepest oceans, highest mountains, tropical rainforests, and polar regions. They occupy nearly every environment on Earth, with high densities reported in mosses and lichens.",Diverse habitats.,True,"Yes, ""Diverse habitats"" is a reasonably acceptable answer to the statement about tardigrades."
5,What is the scientific significance of tardigrades?,Tardigrades have implications in astrobiology and biotechnology research. Their ability to withstand extreme conditions can provide insights into survival mechanisms in extraterrestrial environments. Tardigrade proteins might also have applications in preserving biological materials and vaccine development.,"vaccine development, biological materials, and extraterrestrial environments.",True,The answer is comprehensive and provides relevant information about tardigrades and their significance in astrobiology and biotechnology.


## Image Generation Wrapper

In [223]:
from IPython.display import Image
from openai import OpenAI

class ImageGenerator:
    def __init__(self, prompt, size="1024x1024", quality="standard", n=1):
        self.prompt = prompt
        self.size = size
        self.quality = quality
        self.n = n
        self.client = OpenAI()

    def generate_image(self):
        response = self.client.images.generate(
            model="dall-e-3",
            prompt=self.prompt,
            size=self.size,
            quality=self.quality,
            n=self.n
        )

        image_url = response.data[0].url
        return Image(url=image_url)

ig = ImageGenerator(prompt="cartoon tardigrade")
image = ig.generate_image()
image


In [209]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-Qp4XY1TyXBguzKUJAPFkO9GH/user-ZST1EW9BfdA7Z3rDWA7nJLTW/img-xdHQ2FbmY8LrWHS2i2kgamaR.png?st=2023-12-17T05%3A10%3A06Z&se=2023-12-17T07%3A10%3A06Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-12-16T19%3A38%3A23Z&ske=2023-12-17T19%3A38%3A23Z&sks=b&skv=2021-08-06&sig=u/MyDWFqjwPpz6kyRuN%2BD2LM5J0Py6W95sBjTPpVoMs%3D'